# SnowflakeConnector Testing Notebook

In [ ]:
# Import required modules and classes
from config import SnowflakeConfig
import pandas as pdr

In [2]:
# Initialize configuration and connector

config = SnowflakeConfig(
    database="PROJECTX",
    schema="PUBLIC",
    search_service="CC_SEARCH_SERVICE_CS",
    warehouse="COMPUTE_WH",
    stage_name="@docs"
)

connector = SnowflakeConnector(config)
print("Connector initialized successfully")

Connection Parameters (excluding password):
Account: lmb53062.us-east-1
User: STEPHANIENGUYEN
Database: PROJECTX
Schema: PUBLIC
Warehouse: COMPUTE_WH
Snowflake session created successfully!
Successfully initialized SnowflakeConnector with search service: CC_SEARCH_SERVICE_CS
Connector initialized successfully


In [3]:
res = connector.get_similar_chunks_search_service(query="Why do people resist switching to alternative opportunities when their current livelihood becomes unsustainable?", num_chunks=3)
res

'{"results":[{"chunk":"informal taxation and incentives for conformity, in the extreme leading to what Acemoglu and Robinson (2019) refer to as the \\\\\\"cage of norms.\\\\\\" An important next step is to incorporate such adverse aspects of community relations and investigate when more favorable employment outcomes support positive community norms and when they instead abet negative or discriminatory norms.Third, our work also points towards new empirical explorations. More work is needed to investigate whether there is a causal relationship between the disappearance of attractive em ployment opportunities for workers and the retrenchment of civic life in local communities, and whether this causal effect works through the mechanisms highlighted by our framework. The evidence presented in, among others, Wilson (1996), Black, McKinnish, and Sanders (2005), and Autor, Dorn, and Hanson (2019) suggests that this is plausible, but the exact causal mech- anisms have yet to be studied empiric

In [19]:
for item in connector.root.databases["PROJECTX"].schemas["PUBLIC"].stages["docs"].list_files():
    print(item)

name='docs/Employment and Community - Socioeconomic Cooperation and Its Breakdown.pdf' size='417598' md5='67c5e9441f90342dec4b80fef92f148d' last_modified='Mon, 20 Jan 2025 21:15:52 GMT'
name='docs/Help_for_the_Heartland_20240815.pdf' size='9707566' md5='262dcba9373b9091c94a03f7a9358e69-2' last_modified='Mon, 20 Jan 2025 21:15:53 GMT'
name='docs/Learning from Ricardo and Thompson - Machinery and Labor in the Early Industrial Revolution - and in the Age of AI.pdf' size='346719' md5='dae373a77eaad42296bad5bd0c5a65bd' last_modified='Mon, 20 Jan 2025 21:15:52 GMT'
name='docs/Mistrust, Misperception, and Misunderstanding - Imperfect Information and Conflict Dynamics.pdf' size='971967' md5='a5631684bb0377e26c7d2d67f88daa4e' last_modified='Mon, 20 Jan 2025 21:15:52 GMT'
name='docs/Tasks at Work - Comparative Advantage, Technology and Labor Demand.pdf' size='5511832' md5='88092466e50d87ccc480ed43e5f2fe9c-2' last_modified='Mon, 20 Jan 2025 21:15:54 GMT'
name='docs/The Simple Macroeconomics of AI

In [ ]:
from utils.snowflake_upload import upload_to_stage


upload_to_stage()

# Test chat

In [ ]:
# #agents initialization
# from assistance.user_proxy import IntentClassifier, UserProxy
# from assistance.web_search_agent import WebSearchAgent
# from assistance.writer_agent import WriterAgent
# from assistance.critics_agent import CriticAgent
# from assistance.documents_reading_agent import DocumentReadingAgent
# from services.search_service import create_prompt, create_web_search_prompt, generate_request_to_recipient, reflection_message


# def search(message: str):
#     #agents initialization
#     user_proxy = UserProxy()
#     web_search_agent = WebSearchAgent()
#     writer_agent = WriterAgent()
#     critic_agent = CriticAgent()
#     document_reading_agent = DocumentReadingAgent()
#     web_search_intent = IntentClassifier()

#     # Sequential chat configuration
#     user_proxy.register_nested_chats(
#         chat_queue= [
#             {
#                 "recipient": critic_agent, 
#                 "clear_history": True,
#                 "message": reflection_message,
#                 "summary_method": "last_msg", 
#                 "max_turns": 1
#             }
#             ],
#         trigger=writer_agent
#     )
    
#     res = document_reading_agent.get_relevant_information(message=message)
#     search_res = res['content'] 
#     print("search_res:", search_res)
#     web_search_prompt = create_web_search_prompt(search_res=search_res, message=message)

#     is_search = web_search_intent.classify(web_search_prompt)
#     print("is_search: ", is_search)
#     is_search = is_search['content']
   

c:\Users\tnguy\hackathon\projectx\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

from utils.agents_utils import search


message = "Find me an article about clime change"
res = search(message=message)
res   

Connection Parameters (excluding password):
Account: lmb53062.us-east-1
User: STEPHANIENGUYEN
Database: PROJECTX
Schema: PUBLIC
Warehouse: COMPUTE_WH
Snowflake session created successfully!
Successfully initialized SnowflakeConnector with search service: CC_SEARCH_SERVICE_CS
search_result [{'chunk': '61-103. KATZ,LAWRENCE F AND KEVIN M MURPHY (1992): \\"Changes in Relative Wages,1963-1987: Supply and Demand factors,\\" The Quarterly Journal of Economics, 107, 35-78. KIKUCHI, SHINNOsUKE (2024): \\"Automation, Fragmentation, and Changes in Comparative Ad- vantage,\\" Tech. rep. KIM, RYAN AND JoNATHAN VoGEL (2021): \\"Trade Shocks and Labor Market Adjustment,\\" American Economic Review: Insights, 3, 115-30. KLENOW,PETER J. AND ANDREs RODRiGUEz-CLARE (1997):\\"The Neoclassical Revival in Growth Economics: Has It Gone Too Far?\\" NBER Macroeconomics Annual, 12, 73-103 KRUSELL, PER, LEE E. OHANIAN, JOSE-ViCTOR RiOS-RULL, AND GIOVANNI L. VIOLANTE (2000): \\"Capital-Skill Complementarity and 

ValidationError: 1 validation error for ChatCompletionMessage
content
  Input should be a valid string [type=string_type, input_value=[TextChunk(text="### Summ...(text='.', type='text')], input_type=list]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type

In [2]:
res[-1].chat_history[-1]['content']

"1. Summary : The strategic competition between the US and China involves economic decoupling, with Vietnam playing a key role by strengthening its economic ties with the US and reducing its reliance on China.\n\n2. Detailed Analysis : The document highlights that Vietnam has strengthened its diplomatic ties with the US, particularly in economic cooperation. This alignment serves the US's strategic interests by providing an alternative market to China. Vietnam has actively sought to diversify its export markets and decrease its dependence on China, which aligns with the US's economic goals in the region. This economic cooperation has made Vietnam a crucial partner for the US in its strategic competition with China.\n\n3. Citations : 1. vietnam-2024-outlook-report-en.pdf"

In [1]:
from services.rag_no_agents import NoAgentRAG
from config import SnowflakeConfig
from services.rag_agents import AgentRAG, FilteredAgentRAG

decorating <function AgentRAG.retrieve at 0x000001940EACD260>
decorating <function AgentRAG.generate_completion at 0x000001940FCE4360>
decorating <function AgentRAG.query at 0x000001940FCE4400>
adding method <class 'services.rag_agents.AgentRAG'> retrieve services.rag_agents
adding method <class 'services.rag_agents.AgentRAG'> generate_completion services.rag_agents
adding method <class 'services.rag_agents.AgentRAG'> query services.rag_agents
🦑 Initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `TruSession` to prevent this.
decorating <function NoAgentRAG.retrieve at 0x000001940FF79C60>
decorating <function NoAgentRAG.generate_completion at 0x000001940FF79940>
decorating <function NoAgentRAG.query at 0x000001940FF79BC0>
adding method <class 'services.rag_no_agents.NoAgentRAG'> retrieve services.rag_no_agents
adding method <class 'services.rag_no_agents.NoAgentRAG'> generate_completion services.rag_n

In [4]:
no_agent_rag = NoAgentRAG(config=SnowflakeConfig())
search_res = no_agent_rag.query(query="How does the distinction between 'easy-to-learn' and 'hard-to-learn' tasks impact the predicted productivity gains from AI in the next decade?")
Markdown(search_res)

Snowflake session created successfully!
Successfully initialized SnowflakeConnector with search service: CC_SEARCH_SERVICE_CS


The distinction between 'easy-to-learn' and 'hard-to-learn' tasks impacts the predicted productivity gains from AI by reducing the estimated Total Factor Productivity (TFP) increase within the next 10 years. When considering only easy tasks, the upper bound for TFP increase is about 0.66%. However, recognizing the presence of hard tasks lowers this upper bound to approximately 0.53%. This adjustment accounts for the difficulty in automating and achieving cost savings in hard tasks, which are more complex and lack clear metrics for success.

Sources:
- The Simple Macroeconomics of AI.pdf

In [2]:
agent_rag = AgentRAG(config=SnowflakeConfig())
search_res = agent_rag.query(query="Based on 'Vietnam 2024 Outlook Report', what are Vietnam’s largest export and import markets, and how have these relationships evolved?")
search_res

Snowflake session created successfully!
Successfully initialized SnowflakeConnector with search service: CC_SEARCH_SERVICE_CS

********************************************************************************
Starting a new chat....

********************************************************************************
user_proxy (to writer_agent):


   Use the context information and relevant documents to answer the question.
   If there is no context information, answer based on your general knowledge. 

   Respond based on the context information provided:
    - If the context information and relevant documents are not sufficient to answer user's message, *ONLY* respond with your general knowledge in any format.
    - If there is relevant information in the context information and relevant documents, *ONLY* respond in this format:
        - Summary: A brief summary of context information and relevant documents.
        - Detailed Analysis: Provide an in-depth and detailed information to an

Token indices sequence length is longer than the specified maximum sequence length for this model (1428 > 512). Running this sequence through the model will result in indexing errors


writer_agent (to user_proxy):

- **Summary**: Vietnam's largest export market is the United States, while its largest import market is China. These trade relationships have evolved due to factors such as trade liberalization, economic reforms, and geopolitical dynamics like the U.S.-China trade dispute.

- **Detailed Analysis**:
  - **Export Market**: The United States is Vietnam's largest export market. This relationship has been strengthened by Vietnam's lower labor costs and the U.S.-China trade dispute, which has led to an increase in Vietnam's exports to the U.S. The electronics and computers sector, driven by Samsung, has emerged as Vietnam's top export, surpassing traditional exports like phones and garments.
  - **Import Market**: China is Vietnam's largest import market. This relationship is crucial for Vietnam's manufacturing sector, as it imports materials and machinery from China. However, the limited growth in imports has posed challenges for export processing.
  - **Evolu

"- **Summary**: Vietnam's largest export market is the United States, while its largest import market is China. These trade relationships have evolved due to trade liberalization, economic reforms, and strategic use of Free Trade Agreements (FTAs).\n\n- **Detailed Analysis**:\n  - **Export Market**: The United States is Vietnam's largest export market. This relationship has been bolstered by Vietnam's strategic use of FTAs and economic reforms, which have opened up new markets and attracted foreign investors.\n  - **Import Market**: China is Vietnam's largest import market. This relationship is vital for Vietnam's manufacturing sector, as it relies on imports of materials and machinery from China. The limited growth in imports has posed challenges, but Vietnam has managed to maintain a trade surplus.\n  - **Evolution of Trade Relationships**:\n    1. **Trade Liberalization and Economic Reforms**: These factors have significantly contributed to the growth of Vietnam's export market, par

In [2]:
from assistance.documents_reading_agent import DocumentReadingAgent
document_reading_agent = DocumentReadingAgent()
agent_rag = AgentRAG(config=SnowflakeConfig())
query = "Based on 'Vietnam 2024 Outlook Report', what are Vietnam’s largest export and import markets, and how have these relationships evolved?"
relev_doc = agent_rag.get_similar_chunks_search_service(query=query)
relevant_chunks = document_reading_agent.get_relevant_information(message=query, retrieve_relevant_documents=relev_doc)

Snowflake session created successfully!
Successfully initialized SnowflakeConnector with search service: CC_SEARCH_SERVICE_CS


In [3]:
from IPython.display import Markdown
Markdown(relevant_chunks)

*Exports:*
- "The U.S. emerged as Vietnam's largest export market..." ('vietnam-2024-outlook-report-en.pdf')
- "Electronics and computers emerged as Vietnam's top export, surpassing phones and garments, driven by Samsung, the country's largest exporter." ('vietnam-2024-outlook-report-en.pdf')

*Imports:*
- "China stood as the largest import market for Vietnam in the first 11 months of 2023." ('vietnam-2024-outlook-report-en.pdf')

*Trade Relationship Evolution:*
- "FTAs continued to open up new markets for manufacturers, leveraging Vietnam's lower labor costs and benefiting from the U.S.- China trade dispute." ('vietnam-2024-outlook-report-en.pdf')
- "The industry's growth was fueled by trade liberalization, economic reforms, and enhanced labor quality, attracting foreign investors to relocate their electronics sector investments to Vietnam." ('vietnam-2024-outlook-report-en.pdf')
- "Vietnam achieved an eighth consecutive year of trade surplus, reaching a record high of Us$683 billion, sustaining robust trade relations with developed and robust markets." ('vietnam-2024-outlook-report-en.pdf')

*Trade Partners:*
- "Top 5 trading partners, 2023 (US$ billion) - Export: U.S., EU, ASEAN, South Korea - Import: China, ASEAN, South Korea, EU, U.S." ('vietnam-2024-outlook-report-en.pdf')

In [3]:
filtered_agent_rag = FilteredAgentRAG(config=SnowflakeConfig())
search_res = filtered_agent_rag.query(query="Based on 'Vietnam 2024 Outlook Report', what are Vietnam’s largest export and import markets, and how have these relationships evolved?")
search_res

Snowflake session created successfully!
Successfully initialized SnowflakeConnector with search service: CC_SEARCH_SERVICE_CS
Filtering guardrail for query ...

********************************************************************************
Starting a new chat....

********************************************************************************
user_proxy (to writer_agent):


   Use the context information and relevant documents to answer the question.
   If there is no context information, answer based on your general knowledge. 

   Respond based on the context information provided:
    - If the context information and relevant documents are not sufficient to answer user's message, *ONLY* respond with your general knowledge in any format.
    - If there is relevant information in the context information and relevant documents, *ONLY* respond in this format:
        - Summary: A brief summary of context information and relevant documents.
        - Detailed Analysis: Provide an in-de

'# # # Summary\nVietnam\'s largest export market is the U.S., while its largest import market is China, based on data from the first 11 months of 2023.\n\n# # # Detailed Analysis\nAccording to the "Vietnam 2024 Outlook Report," the U.S. is Vietnam\'s largest export market, with exports valued at approximately $100.26 billion. Meanwhile, China is the largest import market for Vietnam, with imports valued at around $119.233 billion. These figures reflect the trade dynamics between Vietnam and its major partners as of the first 11 months of 2023. However, the report does not provide detailed historical data to illustrate how these relationships have evolved over time.\n\n# # # Source\n- vietnam-2024-outlook-report-en.pdf'